In [2]:
from pymongo import MongoClient
import logging

# Configure logging
logging.basicConfig(filename='analysis_log.txt', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

client = MongoClient("mongodb://localhost:27017/")
db = client.bachelor

reports_count = db.reports.count_documents({})
packages_count = db.packages.count_documents({})
# Wie viele Berichte gibt es?
print(f"Anzahl der Berichte: {reports_count}")
# Wie viele Pakete gibt es?
print(f"Anzahl der Pakete: {packages_count}")

Anzahl der Berichte: 7709
Anzahl der Pakete: 14257


In [11]:
# Beliebtste Helm Charts

helm_charts = {}

for package in db.packages.find({}):
    
    if 'security_report_summary' in package:
        total_vulnerabilities = 0
        for severity in package['security_report_summary'].values():
            total_vulnerabilities += severity
    else :
        total_vulnerabilities = 999999

    subscriptions = int(package["stats"]["subscriptions"])


    scan_report = db.configscan.find_one({"name": package['name'], "version": package['version']})

    if scan_report is not None:
        scan = scan_report["linter_output"]["Reports"]
        if scan is not None:
            misconfig_counter = len(scan)
        else:
            misconfig_counter = 0
    else:
        misconfig_counter = 999999

    package_name = package['name']
    repository_name = package["repository"]["name"]
    name_string = f"{repository_name}/{package_name}"

    helm_charts[name_string] = {
        'vulnerabilities': total_vulnerabilities,
        'subscriptions': subscriptions,
        'misconfigurations': misconfig_counter
    }

# Sort the helm charts by subscriptions
sorted_helm_charts = sorted(helm_charts.items(), key=lambda item: item[1]['subscriptions'], reverse=True)

print("Beliebteste 20 Helm Charts:")
for chart, data in sorted_helm_charts[:40]:
    print(f"{chart}: {data['subscriptions']} Abonnements, {data['vulnerabilities']} Schwachstellen, {data['misconfigurations']} Fehlkonfigurationen")

Beliebteste 20 Helm Charts:
cert-manager/cert-manager: 561 Abonnements, 0 Schwachstellen, 999999 Fehlkonfigurationen
ingress-nginx/ingress-nginx: 524 Abonnements, 0 Schwachstellen, 999999 Fehlkonfigurationen
prometheus-community/kube-prometheus-stack: 407 Abonnements, 27 Schwachstellen, 999999 Fehlkonfigurationen
argo/argo-cd: 384 Abonnements, 110 Schwachstellen, 999999 Fehlkonfigurationen
grafana/grafana: 268 Abonnements, 1 Schwachstellen, 999999 Fehlkonfigurationen
metrics-server/metrics-server: 181 Abonnements, 11 Schwachstellen, 999999 Fehlkonfigurationen
bitnami/redis: 179 Abonnements, 112 Schwachstellen, 999999 Fehlkonfigurationen
grafana/loki: 170 Abonnements, 999999 Schwachstellen, 999999 Fehlkonfigurationen
traefik/traefik: 164 Abonnements, 0 Schwachstellen, 999999 Fehlkonfigurationen
prometheus-community/prometheus: 164 Abonnements, 17 Schwachstellen, 999999 Fehlkonfigurationen
aws/aws-load-balancer-controller: 149 Abonnements, 999999 Schwachstellen, 999999 Fehlkonfiguratione

In [15]:
# find external-dns/external-dns reports
external_dns = []
reports = db.reports.find({})
for report in reports:
    for key in report:
        if key == '_id':
            continue
        elif 'external-dns/external-dns' in key:
            external_dns.append(report[key])
            break
        



print("\nBerichte für external-dns/external-dns:")
for report in external_dns:
    print("One Report found")


Berichte für external-dns/external-dns:
One Report found
One Report found
One Report found
One Report found
One Report found


In [16]:
for report in external_dns:
    print("report: ", report)

report:  {'Results': [{'Type': 'alpine', 'Class': 'os-pkgs', 'Target': 'k8s.gcr.io/external-dns/external-dns:v0.10.2 (alpine 3.14.3)', 'Packages': [{'ID': 'alpine-baselayout@3.2.0-r16', 'Name': 'alpine-baselayout', 'Layer': {'DiffID': 'sha256:1a058d5342cc722ad5439cacae4b2b4eedde51d8fe8800fcf28444302355c16d', 'Digest': 'sha256:97518928ae5f3d52d4164b314a7e73654eb686ecd8aafa0b79acd980773a740d'}, 'SrcName': 'alpine-baselayout', 'Version': '3.2.0-r16', 'Licenses': ['GPL-2.0'], 'DependsOn': ['busybox@1.33.1-r6', 'musl@1.2.2-r3'], 'SrcVersion': '3.2.0-r16'}, {'ID': 'alpine-keys@2.4-r0', 'Name': 'alpine-keys', 'Layer': {'DiffID': 'sha256:1a058d5342cc722ad5439cacae4b2b4eedde51d8fe8800fcf28444302355c16d', 'Digest': 'sha256:97518928ae5f3d52d4164b314a7e73654eb686ecd8aafa0b79acd980773a740d'}, 'SrcName': 'alpine-keys', 'Version': '2.4-r0', 'Licenses': ['MIT'], 'SrcVersion': '2.4-r0'}, {'ID': 'apk-tools@2.12.7-r0', 'Name': 'apk-tools', 'Layer': {'DiffID': 'sha256:1a058d5342cc722ad5439cacae4b2b4eedde5

In [6]:
# find external-dns/external-dns package details

external_dns = []

for package in db.packages.find({"name": "external-dns", "repository.name": "external-dns"}):
    package_name = package['name']
    repository_name = package["repository"]["name"]
    name_string = f"{repository_name}/{package_name}"

    external_dns.append(package)

for package in external_dns:
    print(package)

{'_id': ObjectId('6842ed2d77368e80731a92b1'), 'package_id': '71e71874-9bda-43c0-899b-46365e61f4e1', 'name': 'external-dns', 'normalized_name': 'external-dns', 'category': 5, 'is_operator': False, 'description': 'ExternalDNS synchronizes exposed Kubernetes Services and Ingresses with DNS providers.', 'logo_image_id': '80e6b87a-544d-4794-aa15-02c73884f9cc', 'keywords': ['kubernetes', 'externaldns', 'external-dns', 'dns', 'service', 'ingress'], 'home_url': 'https://github.com/kubernetes-sigs/external-dns/', 'readme': '# external-dns\n\n![Version: 1.16.1](https://img.shields.io/badge/Version-1.16.1-informational?style=flat-square) ![Type: application](https://img.shields.io/badge/Type-application-informational?style=flat-square) ![AppVersion: 0.16.1](https://img.shields.io/badge/AppVersion-0.16.1-informational?style=flat-square)\n\nExternalDNS synchronizes exposed Kubernetes Services and Ingresses with DNS providers.\n\n**Homepage:** <https://github.com/kubernetes-sigs/external-dns/>\n\n##

In [3]:
#find scan reorts for external-dns/external-dns
external_dns = db.configscan.find_one({"name": "external-dns", "version": "1.16.1", "repository": "external-dns"})

print(external_dns)

{'_id': ObjectId('6851b5349b601bbfe4cd1ab2'), 'name': 'external-dns', 'version': '1.16.1', 'repository': 'external-dns', 'linter_output': {'Reports': [{'Diagnostic': {'Message': 'container "external-dns" has cpu request 0'}, 'Check': 'unset-cpu-requirements', 'Remediation': 'Set CPU requests for your container based on its requirements. Refer to https://kubernetes.io/docs/concepts/configuration/manage-resources-containers/#requests-and-limits for details.', 'Object': {'Metadata': {'FilePath': '/home/ubuntu/bachelor/helm-chart-analysis/Scrape/external-dns/templates/deployment.yaml'}, 'K8sObject': {'Namespace': 'default', 'Name': 'test-release-external-dns', 'GroupVersionKind': {'Group': 'apps', 'Version': 'v1', 'Kind': 'Deployment'}}}}, {'Diagnostic': {'Message': 'container "external-dns" has memory limit 0'}, 'Check': 'unset-memory-requirements', 'Remediation': 'Set memory limits for your container based on its requirements. Refer to https://kubernetes.io/docs/concepts/configuration/ma